In [1]:
# CMSE Project
import os
print(os.getcwd())
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import matplotlib.pyplot as plt
import random as random
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf

C:\Users\Matt\Downloads\CMSE project


In [2]:
# Imports functions from seperate notebook
%run Functions.ipynb

Here I load in the data and spit out some summary statistics.

In [3]:
# Reading in Data
# Keys to extract formatted State and County names
key = pd.read_excel('fips_codes_website.xls')   # Counties and major cities
key00 = pd.read_csv('FIPS_countycodes.txt',delimiter=',')  # Counties only
key00['GU Name'] = key00['GU Name'].str[:-7]  # Slice 'county' from names
# Housing Data
zillow2019 = pd.read_excel("County_Zhvi_Summary_AllHomes2019.xls")
zillow2019['RegionName'] = zillow2019['RegionName'].str[:-7]  # Slice 'county' from names
# Crime
crime2010 = pd.read_csv('33523-0001-crimedata2010.txt',delimiter='\t')
crime2012 = pd.read_csv('35019-0001-crimedata2012.txt',delimiter='\t')
crime2014 = pd.read_csv('36399-0001-crimedata2014.txt',delimiter='\t')
crime2016 = pd.read_csv('37059-0001-crimedata2016.txt',delimiter='\t')
# Enumployment & Wages
#test_wages_18 = pd.read_excel('allhlcn183.xlsx',sheet_name='US_St_Cn_MSA')
wage_unemp2018 = pd.read_csv('allhlcn183wage_unemp2018.txt',delimiter='\t',dtype={'St':str})
wage_unemp2016 = pd.read_csv('allhlcn163wage_unemp2016.txt',delimiter='\t',dtype={'St':str})
wage_unemp2014 = pd.read_csv('allhlcn143wage_unemp2014.txt',delimiter='\t',dtype={'St':str})
wage_unemp2012 = pd.read_csv('allhlcn123wage_unemp2012.txt',delimiter='\t',dtype={'St':str})
wage_unemp2010 = pd.read_csv('allhlcn103wage_unemp2010.txt',delimiter='\t',dtype={'St':str})

print("2014 crime:",crime2014.shape,"// key:",key.shape)
print("2019 Housing:",zillow2019.shape)
print('2018 Wages:',wage_unemp2018.shape)
x = ((crime2014.shape[0])*(crime2014.shape[1]))*4 + ((wage_unemp2018.shape[0])*(wage_unemp2018.shape[1]))*5 + (key00.shape[0])*key00.shape[1]
print(x,'unique datapoints in all datasets')

2014 crime: (3177, 56) // key: (41787, 7)
2019 Housing: (1921, 17)
2018 Wages: (62724, 21)
7313843 unique datapoints in all datasets


Here is where I do the bulk of my data cleaning. 

I found it difficult to eliminate the copyset errors because the wage_unemployment dataframes are coded with key values in the County and State columns (as seen above). There is national, state, and metropolitan area data in these dataframes. But the values that I would need to concat/merge on are 'NaN' for these other regional divisions. So I am not able to merge with the dataframe as is, and dropping the problem rows then merging produces the error. 

The rest of my data cleaning is just converting columns of numbers from strings into integers. The BLS also started a new method of how they code in the various industries within a region, so I also have to clean that up to match past years. 


The functions called in this cell are imported from another notebook.

In [4]:
# Data Cleaning #
# Yearly cleaned databases
crime10 = crime_cleaner(crime2010)
crime12 = crime_cleaner(crime2012)
crime14 = crime_cleaner(crime2014)
crime16 = crime_cleaner(crime2016)
cols = list(crime14.columns.values) # Grab columns
# Cleaned Wage Data
economic10 = wage_cleaner(wage_unemp2010) # Returns 3 items; county, state, national, and urban level data sets
economic12 = wage_cleaner(wage_unemp2012)
economic14 = wage_cleaner(wage_unemp2014)
economic16 = wage_cleaner(wage_unemp2016)
economic18 = wage_cleaner(wage_unemp2018)
#wage18[0]['Industry'] = [i[i.find(' ')+1:] for i in wage18[0]['Industry']] # Get to work
economic18['Industry'] = economic18['Industry'].str[1:] # Remove first space in 2018 industry codes

# Combined crime and economic dataframes and rearrange columns
db2010 = crime10.merge(economic10,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2010 = db2010[['Year','State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY','Area Type','St Name','Area','Ownership','Industry','Establishment Count','July Employment','August Employment','September Employment','Total Quarterly Wages','Average Weekly Wage','Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]
db2012 = crime12.merge(economic12,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2012 = db2012[['Year','State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY','Area Type','St Name','Area','Ownership','Industry','Establishment Count','July Employment','August Employment','September Employment','Total Quarterly Wages','Average Weekly Wage','Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]
db2014 = crime14.merge(economic14,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2014 = db2014[['Year','State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY','Area Type','St Name','Area','Ownership','Industry','Establishment Count','July Employment','August Employment','September Employment','Total Quarterly Wages','Average Weekly Wage','Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]
db2016 = crime16.merge(economic16,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2016 = db2016[['Year','State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY','Area Type','St Name','Area','Ownership','Industry','Establishment Count','July Employment','August Employment','September Employment','Total Quarterly Wages','Average Weekly Wage','Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]

C:\Users\Matt\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Matt\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Matt\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

Here is some code to randomly pull some counties from my database. There is no data attached to them, useful to find random unknown counties to search/test against.

In [5]:
# Generate Sample of Randomly Selected Counties
n = 5 # Numer of random samples
print("Randomly Generated Cities:\nState County")
for i in range(n):
    r = random.randint(0,len(crime12))
    print(crime12['State Abbreviation'].iloc[r]+', '+crime12['GU Name'].iloc[r])

Randomly Generated Cities:
State County
MN, Pennington
VA, Norfo
VA, Norfo
MS, Lincoln
OR, Columbia


Here is a search function that will find all available data for any single county. This cell needs to be run for all other cells to work.

**Random Database of Counties Generation**

Here is where I actualy generate a database of random counties and all associated information. The cells immediatly following will automatically plot and display info on whatever single county you searched for. While not specifically useful for analysis I encourage you to browse through them simply to see what variables I have in my data. Otherwise this part can be skipped to where I have a bolded headling about regressions.

Each time this cell is ran an entirely new database is generated.

In [6]:
# Selects 10 Random counties and stores them as a dictionary by count // Key = MI, County
samp_db = {}
for n in range(10):
    r = random.randint(0,len(crime12))
    x = crime12['State Abbreviation'].iloc[r] # Pull State Abreviation
    y = crime12['GU Name'].iloc[r]  # Pull analogous county name
    z = x+', '+y # Format name for dictionary key
    agg = search(y,x) # Return criminal and economic data
    samp_db[z] = agg  # Add data to empty dictionary
keys = list(samp_db.keys())
print(samp_db.keys())

dict_keys(['GA, Taylor', 'VA, Buchanan', 'FL, Palm Beach', 'MN, Stevens', 'OK, Dewey', 'ME, Kennebec', 'NE, Colfax', 'VA, Fauquier', 'MO, Pemiscot', 'KY, Lyon'])


Here is a function to clean the crime data and store it into a 'crime dictionary'. Will work if you update the 'crimes' variable at the top with new crimes to pull.

Here I wrote a function to clean and store the industry data into a dictionary as well as a function to pull the 4 largest industries in our county. Although this is only ran with the individual county data (as opposed to finding the largest industries across the sample database, it shouldn't matter much since the most popular industries do not vary much from county to county).

In [7]:
industry = most_establishments(samp_db[keys[1]][3])  # Returns 4 largest industries in 2016 ,index = 3
industrydata = industry # Data set for other metrics in largest industries
industry = list(industry['Industry']) # List of largest industries for reference purposes

**REGRESSIONS**

Here I figured out how to [sort of] do multiple regression for a single county. It runs, however the numbers are junk, as indicated by the near perfect R-squared.

Here I wrote a couple functions specifically designed to clean and format data for multiple counties to be easily stored inside a dataframe.

In [8]:
samp_dic = {}
stat = 'Establishment Count','Average Weekly Wage','September Employment'
for s in stat:
    for n in industry:
        samp_dic[n+' '+s] = industrydata_cleaner(samp_db,s,n)
        
crimes = samp_db[keys[1]][0].columns[:48] # Reset crimes to grab all crime data
for i in crimes:
    samp_dic[i] = sample_crime_cleaner(samp_db,i)
finaldb = pd.DataFrame(data=samp_dic)
# Reorganize columns
cols = finaldb.columns.tolist()
cols = cols[12:15]+cols[:12]+cols[15:]
finaldb = finaldb[cols]
print("Counties in Sample Data: ",samp_db.keys(),'\n')
#print("Columns in final database: ",finaldb.columns)
print('\nFinal Database for all 10 counties over 4 years in my sample.')
finaldb

Counties in Sample Data:  dict_keys(['GA, Taylor', 'VA, Buchanan', 'FL, Palm Beach', 'MN, Stevens', 'OK, Dewey', 'ME, Kennebec', 'NE, Colfax', 'VA, Fauquier', 'MO, Pemiscot', 'KY, Lyon']) 


Final Database for all 10 counties over 4 years in my sample.


,Year,State Abbreviation,GU Name,Service-providing Establishment Count,Education and health services Establishment Count,"Trade, transportation, and utilities Establishment Count",Goods-producing Establishment Count,Service-providing Average Weekly Wage,Education and health services Average Weekly Wage,"Trade, transportation, and utilities Average Weekly Wage",...,OFAGFAM,DUI,LIQUOR,DRUNK,DISORDR,VAGRANT,ALLOTHR,SUSPICN,CURFEW,RUNAWAY
0,2010,GA,Taylor,[104],[13],[35],[21],[644],[422],[683],...,2,32,0,0,10,0,22,8,0,0
1,2012,GA,Taylor,[110],[14],[40],[19],[630],[517],[734],...,1,19,1,0,11,0,17,4,0,0
2,2014,GA,Taylor,[115],[16],[38],[21],[673],[526],[795],...,3,18,1,0,25,0,49,0,0,1
3,2016,GA,Taylor,[114],[17],[38],[25],[777],[598],[858],...,1,12,1,0,13,1,73,0,0,0
4,2010,VA,Buchanan,[422],[107],[157],[107],[562],[645],[589],...,1,108,21,108,16,0,773,0,0,0
5,2012,VA,Buchanan,[428],[111],[157],[113],[582],[713],[566],...,0,74,15,106,1,0,745,0,0,0
6,2014,VA,Buchanan,[417],[121],[140],[92],[573],[711],[549],...,0,78,12,57,1,0,852,0,0,0
7,2016,VA,Buchanan,[452],[138],[131],[83],[582],[742],[529],...,0,50,0,59,1,0,621,0,0,0
8,2010,FL,Palm Beach,[41804],[5531],[9229],[6103],[824],[870],[721],...,0,0,0,0,0,0,0,0,0,0
9,2012,FL,Palm Beach,[43558],[5889],[9417],[5934],[845],[892],[729],...,0,0,0,0,0,0,0,0,0,0


Here is a regression of the number of establishments in each of the largest industries on drug possession arrests.

In [ ]:
x = finaldb[list(finaldb.columns.values[3:7])]  # Use only the first 3 columns to regress
y = finaldb['DRGPOSS']  # Set y variable

model = sm.OLS(y, x).fit()
predictions = model.predict(x) # make the predictions by the model

print(model.summary())